In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np

import czifile
from pathlib import Path
import os
from skimage import morphology, exposure, filters
from skimage.feature import canny
from skimage.measure import regionprops, regionprops_table, label
from scipy import ndimage, misc

In [ ]:
# replace with a configuration file
work_dir = '/Users/robinhood/Dropbox (HMS)/Data/imaging/processing'

channel_dict = {'DAPI': 0, 'TL':1, 'gt':2, 'pho':3, 'hb':4}
bkgd_signal = 150
file_name = 'wt_20210510_gthb_5.czi'
channel_list = ['DAPI', 'gt', 'pho', 'hb'] #keep for iteration
shape_channel = 'DAPI'
age_channel = 'DAPI'
ap_channel = 'DAPI'
dv_channel = 'hb'
kd_channel = 'pho'
out_channels = ['gt', 'hb']

In [ ]:
directory = work_dir
for filename in os.listdir(directory):
    if filename.endswith(".czi"):
        print(os.path.join(directory, filename))
        path = os.path.join(directory, filename)
        load_embryo(path, filename) # only do this once. 
        get_dimensions(img, z_plane) # only do this once to get all the general dimensions
        ...
        # repeat with z_plane = z_plane -3 / z_plane = z_plane + 3
        
    else:
        continue

In [ ]:
# make this its own script like get_Age

def load_embryo(config) # ask Jeff how to write the proper syntax here.  
    czi = czifile.imread(file_name)
    img=czi.squeeze()
    data = {}
    for channel in channel_list:
        data[channel] = img[channel_dict[channel],...]

    # get dimensions and the central z-plane
    xdim = img.shape[2]
    ydim = img.shape[3]
    z_plane = img.shape[1] / 2
    z_plane = round(z_plane)

 

In [ ]:
# make this its own script like get_Age

def get_dimensions(img, z_plane)

    max_AP = data[ap_channel].max(0) # this will be how to pick AP axis
    z_plane_shape = data[shape_channel][z_plane,:,:]
    z_plane_dv = data[dv_channel][z_plane,:,:]

    z_plane_normalized = exposure.adjust_gamma(z_plane_shape)
    z_hyst_shape = filters.apply_hysteresis_threshold(z_plane_normalized, bkgd_signal, np.max(z_plane_normalized)-1)
    zclosed_v1 = morphology.binary_closing(z_hyst_shape)
    zedge = canny(zclosed_v1, sigma=2.0, low_threshold=0.55, high_threshold=0.8) 

    # get the rotation angle for the embryo at mid plane

    emb_label= label(zclosed_v1.astype(np.uint8))
    emb_regions = regionprops(zclosed_v1.astype(np.uint8))

    for props in emb_regions:
        y0, x0 = props.centroid
        orientation = props.orientation
        long_axis = props.major_axis_length
        short_axis = props.minor_axis_length
        minr, minc, maxr, maxc = props.bbox

        x1 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length #top
        y1 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
        x3 = x0 - math.cos(orientation) * 0.5 * props.minor_axis_length #bottom
        y3 = y0 + math.sin(orientation) * 0.5 * props.minor_axis_length
        
        x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
        y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length
        x4 = x0 + math.sin(orientation) * 0.5 * props.major_axis_length
        y4 = y0 + math.cos(orientation) * 0.5 * props.major_axis_length

    # print an image to save

    # make a better mask
    footprint=morphology.disk(25)
    dil1 = morphology.binary_dilation(zedge, footprint)
    dilated_zedge_shape = morphology.binary_dilation(zedge) # fills a hole in edges
    zclosed = morphology.flood(dilated_zedge_shape, (int(y0), int(x0)))

In [ ]:
# make this its own script like get_Age
# Ask Jeff about how to best write the config file such that the age and AP are included. 

def get_AP()
    rotation_axis = 90 - (orientation * 180/math.pi)
    rotated_embshape = ndimage.rotate(zclosed, rotation_axis, reshape=False)

    os.makedirs(path[:-4], exist_ok=True)
    fig.savefig(os.path.join(path[:-4], f"{filename[:-4]}_AP"))
    plt.imshow(rotated_embshape)

In [ ]:
def rotate_emb(zclosed, config):
    rotation_axis = 90 - (orientation * 180/math.pi)
    rotated_embshape = ndimage.rotate(zclosed, rotation_axis, reshape=False)
                                        # WRITE IN A/P IF/ELSE FOR PROPER BEAN FLICKING
    flip = True
    if flip:
        rotated_embshape= np.fliplr(rotated_embshape)

    #series of commands to make a nice emb shape again
    rotated_embshape = exposure.adjust_gamma(rotated_embshape)
    rotated_embshape = filters.apply_hysteresis_threshold(rotated_embshape, bkgd_signal, np.max(rotated_embshape)-1)
    rotated_embshape = morphology.binary_closing(rotated_embshape)
    rotated_embshape = canny(rotated_embshape, sigma=2.0, low_threshold=0.55, high_threshold=0.8) 
    rotated_embshape = morphology.binary_dilation(rotated_embshape) # fills a hole in edges
    rotated_embshape = morphology.flood(rotated_embshape, (int(y0), int(x0)))

    fig, ax = plt.subplots(figsize=(10,10))
    plt.imshow(rotated_embshape, cmap=plt.cm.gray)  


    #rotate embryo shape
    rot_zedge = canny(rotated_embshape, sigma=2.0, low_threshold=0.55, high_threshold=0.8) 

    os.makedirs(path[:-4], exist_ok=True)
    fig.savefig(os.path.join(path[:-4], f"{filename[:-4]}_RotCheck"))
    plt.imshow(rot_zedge)

In [ ]:
#could do the first part as a part of get_dimensions (call get_dimensions again...or just let is be inelegant)

def get_DVknife(rotated_embshape)
    emb_label= label(rotated_embshape.astype(np.uint8))
    emb_regions = regionprops(rotated_embshape.astype(np.uint8))

    for props in emb_regions:
        y0, x0 = props.centroid
        orientation_rot = props.orientation
        long_axis = props.major_axis_length
        short_axis = props.minor_axis_length
        minr, minc, maxr, maxc = props.bbox

        x1 = x0 + math.cos(orientation) * 0.5 * props.minor_axis_length #top
        y1 = y0 - math.sin(orientation) * 0.5 * props.minor_axis_length
        x3 = x0 - math.cos(orientation) * 0.5 * props.minor_axis_length #bottom
        y3 = y0 + math.sin(orientation) * 0.5 * props.minor_axis_length
        
        x2 = x0 - math.sin(orientation) * 0.5 * props.major_axis_length
        y2 = y0 - math.cos(orientation) * 0.5 * props.major_axis_length
        x4 = x0 + math.sin(orientation) * 0.5 * props.major_axis_length
        y4 = y0 + math.cos(orientation) * 0.5 * props.major_axis_length


    major_axis_line = np.zeros((ydim, xdim), dtype=bool)
    m_major = (y2 - y0) / (x2 - x0)
    b_major = (y2 - (m_major * x2))

    # major axis loop
    major_inds = [] # list
    for x in range(major_axis_line.shape[0]):
        y = ((m_major * x) + b_major)
        if y < major_axis_line.shape[0] and y >= 0:
            major_axis_line[int(y),int(x)] = True
            major_inds.append([int(y), int(x)])


    footprint=morphology.disk(10)

    DV_knife = morphology.binary_dilation(major_axis_line, footprint)

In [ ]:
def get_DV()
    footprint=morphology.disk(25)
    dil = morphology.binary_dilation(rot_zedge, footprint)

    # makes a better oval from edges
    #dilated_edge = morphology.binary_dilation(rot_edge_slice) # fills a hole in edges
    #better_closed = morphology.flood(dilated_edge, (int(y0), int(x0)))

    zorro = rotated_embshape*dil # slices off all the extra in dil 1 to make a mask of the area of interest in the embryo
    dv_divide = np.invert(DV_knife) * zorro # cuts the zorro mask with the knife

    dv_label = label(dv_divide)
    dv_regions = regionprops(dv_label)
    dv_props = regionprops_table(dv_label, properties=('area', 'perimeter', 'bbox'))

    blank = np.zeros((ydim, xdim), dtype=bool)
    sideA = blank.copy()
    bbox = dv_regions[0].bbox
    sideA[bbox[0]:bbox[2], bbox[1]:bbox[3]] = dv_regions[0].image

    sideB = blank.copy()
    bbox = dv_regions[1].bbox
    sideB[bbox[0]:bbox[2], bbox[1]:bbox[3]] = dv_regions[1].image

    # use the two sides and mask the snail hysterics image


    decide_A =  hyst_zdv*sideA
    decide_B =  hyst_zdv*sideB

    # decision point for the DV axis

    if np.sum(decide_A) > np.sum(decide_B):
        dorsal = sideB
        dorsal_check = decide_B
        garbage = decide_A
    else:
            dorsal = sideA
            dorsal_check = decide_A
            garbage = decide_B

# add a save figure here to be able to reference back.
    fig, ax = plt.subplots(1,3, figsize=(25,25))
    ax[0].imshow(dorsal)
    ax[1].imshow(dorsal_check, cmap=plt.cm.gray)
    ax[2].imshow(garbage, cmap=plt.cm.gray)